# RNN Implementation and Experiments

This notebook demonstrates the RNN implementation and experiments for the IF3270 Machine Learning assignment. We'll work with the NusaX-Sentiment dataset to perform text classification.

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, SimpleRNN, Dropout, Dense, Bidirectional
from sklearn.metrics import f1_score, classification_report

# Add the parent directory to the path to import our modules
sys.path.append('..')

from src.RNN.nusax_loader import NusaXLoader
from src.RNN.experiments import RNNExperiments
from src.RNN.rnn_model import RNNModel
from src.RNN.layers.rnn_layer import RNNLayer
from src.RNN.layers.embedding_layer import EmbeddingLayer
from src.Base.layers.dense_layer import DenseLayer
from src.Base.layers.dropout_layer import DropoutLayer
from src.Base.layers.activation_layer import Softmax
from src.Base.utils.evaluation import compare_keras_vs_scratch
from src.Base.utils.visualization import plot_training_history

## 1. Load and Explore the NusaX-Sentiment Dataset

In [ ]:
# Initialize the data loader
data_loader = NusaXLoader(batch_size=32)

# Get a batch of training data for visualization
train_dataset = data_loader.get_dataset('train')
for tokens, labels in train_dataset.take(1):
    sample_tokens = tokens.numpy()
    sample_labels = labels.numpy()
    break

# Get vocabulary
vocab = data_loader.get_vocabulary()
print(f"Vocabulary size: {len(vocab)}")
print(f"First 20 words in vocabulary: {vocab[:20]}")

# Decode a few examples
print("\nSample texts:")
for i in range(3):
    # Convert token IDs back to words
    words = [vocab[idx] if idx < len(vocab) else "[UNK]" for idx in sample_tokens[i] if idx > 0]
    text = " ".join(words)
    print(f"Text {i+1}: {text}")
    print(f"Label: {sample_labels[i]}\n")

## 2. Hyperparameter Experiments with RNN

We'll run experiments to analyze the impact of different hyperparameters on RNN performance.

In [ ]:
# Initialize the experiments class
experiments = RNNExperiments(batch_size=32, epochs=10, embedding_dim=100)

### 2.1 Experiment: Number of RNN Layers

In [ ]:
# Define variants for number of RNN layers
layer_count_variants = [
    (1, "1 RNN Layer"),
    (2, "2 RNN Layers"),
    (3, "3 RNN Layers")
]

# Run experiment
layer_count_models, layer_count_histories = experiments.run_layer_count_experiment(layer_count_variants)

### 2.2 Experiment: Number of RNN Cells per Layer

In [ ]:
# Define variants for cell counts
cell_count_variants = [
    ([64], "64 Cells"),
    ([128], "128 Cells"),
    ([256], "256 Cells")
]

# Run experiment
cell_count_models, cell_count_histories = experiments.run_cell_count_experiment(cell_count_variants)

### 2.3 Experiment: RNN Direction

In [ ]:
# Define variants for RNN direction
direction_variants = [
    (False, "Unidirectional RNN"),
    (True, "Bidirectional RNN")
]

# Run experiment
direction_models, direction_histories = experiments.run_direction_experiment(direction_variants)

## 3. From-Scratch RNN Implementation

Now we'll implement the RNN forward propagation from scratch and compare it with the Keras implementation.

### 3.1 Choose and Save a Keras Model

In [ ]:
# Choose one of the trained models (e.g., from the bidirectional experiment)
keras_model = direction_models[1][0]  # Bidirectional RNN model

# Summary of the chosen model
keras_model.summary()

# Save the model weights
keras_model.save_weights('rnn_model_weights.h5')

### 3.2 Create From-Scratch RNN Model

In [ ]:
# Get embedding dimension and vocabulary size
embedding_dim = 100
vocab_size = len(data_loader.get_vocabulary())
hidden_dim = 128
num_classes = data_loader.num_classes
sequence_length = data_loader.max_sequence_length

# Create a from-scratch RNN model that matches the Keras model
scratch_model = RNNModel()

# Add layers corresponding to the Keras model architecture
scratch_model.add(EmbeddingLayer(input_dim=vocab_size, output_dim=embedding_dim))
scratch_model.add(RNNLayer(input_dim=embedding_dim, hidden_dim=hidden_dim, bidirectional=True))
scratch_model.add(DropoutLayer(dropout_rate=0.2))
scratch_model.add(DenseLayer(input_dim=hidden_dim*2, output_dim=num_classes, activation=Softmax()))

### 3.3 Load Weights from Keras Model

In [ ]:
# Load weights from the Keras model
scratch_model.load_weights_from_keras(keras_model)

print("Weights loaded from Keras model to from-scratch implementation.")

### 3.4 Compare Predictions from Keras and From-Scratch Models

In [ ]:
# Get test data
x_test, y_test = data_loader.get_vectorized_data('test')

# Compare predictions
comparison = compare_keras_vs_scratch(keras_model, scratch_model, x_test, y_test, batch_size=32)

print("\nKeras Model Metrics:")
print(f"Accuracy: {comparison['keras_metrics']['accuracy']:.4f}")
print(f"Macro F1-Score: {comparison['keras_metrics']['macro_f1']:.4f}")

print("\nFrom-Scratch Model Metrics:")
print(f"Accuracy: {comparison['scratch_metrics']['accuracy']:.4f}")
print(f"Macro F1-Score: {comparison['scratch_metrics']['macro_f1']:.4f}")

print(f"\nModel Agreement: {comparison['model_agreement']:.4f}")

### 3.5 Visualize Predictions on Sample Texts

In [ ]:
# Get a few test samples
num_samples = 5
sample_indices = np.random.choice(x_test.shape[0], num_samples, replace=False)
sample_texts = x_test[sample_indices]
sample_labels = y_test[sample_indices]

# Make predictions with both models
keras_preds = np.argmax(keras_model.predict(sample_texts), axis=1)
scratch_preds = scratch_model.predict(sample_texts)

# Define sentiment labels
sentiment_labels = ["Negative", "Neutral", "Positive"]

# Visualize the results
for i in range(num_samples):
    # Convert token IDs back to words
    words = [vocab[idx] if idx < len(vocab) else "[UNK]" for idx in sample_texts[i] if idx > 0]
    text = " ".join(words)
    
    # Show true label and predictions
    keras_correct = keras_preds[i] == sample_labels[i]
    scratch_correct = scratch_preds[i] == sample_labels[i]
    
    print(f"\nText: {text[:100]}{'...' if len(text) > 100 else ''}")
    print(f"True sentiment: {sentiment_labels[sample_labels[i]]}")
    print(f"Keras prediction: {sentiment_labels[keras_preds[i]]} {'✓' if keras_correct else '✗'}")
    print(f"Scratch prediction: {sentiment_labels[scratch_preds[i]]} {'✓' if scratch_correct else '✗'}")

## 4. Analysis and Conclusions

Based on our experiments, we can draw the following conclusions about RNN hyperparameters:

### 4.1 Effect of Number of RNN Layers

[Placeholder for your analysis]

### 4.2 Effect of RNN Cell Count per Layer

[Placeholder for your analysis]

### 4.3 Effect of Bidirectional vs Unidirectional RNN

[Placeholder for your analysis]

### 4.4 From-Scratch Implementation Analysis

[Placeholder for your analysis of the from-scratch implementation compared to Keras]